Import packages.

In [ ]:
%matplotlib inline
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile
import numpy as np
import math
from datetime import timedelta
from operator import attrgetter

Specify target populations.

In [ ]:
target = [
    'Rovinj',
    'Valsaline',
    'Sarkodana',
    'Cres',
    'Krk_neu',
    'Krk_Baska',
    'Klenovica',
    'Rab',
    'HafenPag_neu',
    'Pag',
    'PagerBucht',
    'Pag_Holger',
    'DugiOtok',
    'Ugljan_neu',
    'Pasman',
    'SibenikDolac_neu',
    'Kornaten_Sued',
    'TrogirCiovoWest',
    'BracSupetarSutivan',
    'BracPovlja',
    'CampAloaBol',
    'HvarBucht4',
    'HvarMlaska',
    'HvarBogomoje',
    'Mljet',
    'Molunat_neu'
]

The next cell will just read the nc files produced by parcels for each location and write out the final coordinates for each of the particles to a tsv file.

In [ ]:
import glob
from netCDF4 import Dataset

#drift duration in days
drift = 20

dictionary = {}
for loc in target:
    dictionary[loc] = {'time':[], 'lat':[], 'lon':[]}
    
#    fh = open(loc+'.data.tsv')

for year in range(2004,2019):
    print (year)
    for nc in sorted(glob.glob(str(year) + "/release-*.nc")):
        print (nc)
        data = Dataset(nc, 'r')
        #the number of particles
        num_locations = int(data.dimensions['traj'].size)
        num_observations = int(data.dimensions['obs'].size)
        for loc_index in range(num_locations):

            # extract release timepoint from filename
            release = np.datetime64("T".join(["-".join(nc.split('T')[0].split("-")[1:]), ":".join(nc.replace(".nc","").split('T')[1].split("-"))]))
            # caculate last timepoint
            last = release + np.timedelta64(drift, 'D')
            dictionary[target[loc_index]]['time'].append(last)
            dictionary[target[loc_index]]['lat'].append(data.variables['lat'][loc_index][-1])
            dictionary[target[loc_index]]['lon'].append(data.variables['lon'][loc_index][-1])

for loc in dictionary:
    print (loc)
    fh = open('data/'+loc+'.data.tsv', 'w')
    for i in range(len(dictionary[loc]['time'])):
        fh.write("%s\t%s\t%s\n" %(dictionary[loc]['time'][i], dictionary[loc]['lat'][i], dictionary[loc]['lon'][i]))
    fh.close()